In [ ]:
from sys import path

from pandas import DataFrame

path.append("../../../")
from componentes.pipeline.pipeline import Pipeline
from componentes.artefato.arquivo import Arquivo
from componentes.container.container_silver import ContainerSilver
from componentes.container.container_gold import ContainerGold
from componentes.container.categoria_de_armazenamento import \
    CategoriaDeArmazenamento
from notebooks.silver.cnae.schema.cnae_schema import get_cnae_silver_schema

In [ ]:
class DimensaoCNAESilverParaGold(Pipeline):
    """
    Pipeline de dados responsável por executar o processamento de artefatos
    de categoria silver para gold gerando a dimensao cnae
    """
    def _transformar_dados(self, dataframe: DataFrame) -> DataFrame:
        """
        Transforma os dados para criar a modelagem de dimensão
        """
        self._logger.log_mensagem(mensagem="Iniciando transformações")
        dataframe = self._selecionar_e_renomear_colunas(dataframe=dataframe)
        dataframe = self._criar_id_cnae(dataframe=dataframe)
        return dataframe
    
    def _selecionar_e_renomear_colunas(self, dataframe: DataFrame) -> DataFrame:
        """
        Seleciona as colunas necessárias do arquivo CNAE silver e renomeia as
        colunas de acordo com a modelagem definida
        """
        self._logger.log_mensagem(mensagem="Selecionando e renomenado colunas")
        return (
            dataframe[["denominacao", "codigo_cnae", "tipo_cnae"]]
            .rename(
                columns={
                    "denominacao": "descricao",
                    "codigo_cnae": "codigo",
                    "tipo_cnae": "tipo"}))
    
    def _criar_id_cnae(self, dataframe: DataFrame) -> DataFrame:
        """
        Criar um novo identificador único para dimensão
        """
        self._logger.log_mensagem(mensagem="Criando coluna id_cnae")
        dataframe["id_cnae"] = dataframe.reset_index().index + 1
        return dataframe

In [ ]:
artefato_de_entrada = Arquivo(
    nome="construcao",
    categoria_de_armazenamento=CategoriaDeArmazenamento.SILVER,
    fonte="cnae",
    schema=get_cnae_silver_schema())
container_de_entrada = ContainerSilver()
artefato_de_saida = Arquivo(
    nome="dimensao_cnae",
    categoria_de_armazenamento=CategoriaDeArmazenamento.GOLD,
    fonte="dimensao")
container_de_saida = ContainerGold()
pipeline = DimensaoCNAESilverParaGold(
    artefato_de_entrada=artefato_de_entrada,
    container_de_entrada=container_de_entrada,
    artefato_de_saida=artefato_de_saida,
    container_de_saida=container_de_saida)
pipeline.executar_etl()